# ALS IPCA Simulation Study

This notebook validates the ALS IPCA estimator on synthetic data generated according to the model in Kelly, Pruitt, Su (2019).

**IPCA Model:**
$$x_{i,t} = \beta_{i,t}' f_t + \varepsilon_{i,t}$$
$$\beta_{i,t} = \Gamma' c_{i,t}$$

**Objectives:**
1. Generate synthetic panel data following the IPCA model
2. Estimate the model using ALS IPCA
3. Visualize convergence behavior
4. Compare estimated vs true parameters

In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from als_ipca import ALSIPCA
from simulate_ipca_data import generate_ipca_data, compute_gamma_error

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Imports successful.")

---
## 1. Simulation Parameters

In [ ]:
# Simulation parameters
N = 200       # Number of entities
T = 200       # Time periods
K = 3         # Latent factors
L = 10        # Observable characteristics
TARGET_R2 = 0.20
SEED = 42

print(f"N = {N} entities")
print(f"T = {T} time periods")
print(f"K = {K} factors")
print(f"L = {L} characteristics")
print(f"Target R² = {TARGET_R2:.0%}")

---
## 2. Generate Synthetic Data

In [ ]:
sim_data = generate_ipca_data(
    N=N, T=T, K=K, L=L,
    target_r2=TARGET_R2,
    seed=SEED
)

print("Data shapes:")
print(f"  Returns X: {sim_data['X'].shape}")
print(f"  Factors: {sim_data['factors'].shape}")
print(f"  Characteristics: {sim_data['characteristics'].shape}")
print(f"  True Gamma: {sim_data['Gamma'].shape}")
print(f"\nCalibrated error std: {sim_data['params']['error_std']:.4f}")

In [ ]:
# Display true Gamma
Gamma_true = sim_data['Gamma']
print("True Gamma matrix (L x K):")
print(np.round(Gamma_true, 4))

---
## 3. Prepare Data for ALS IPCA

The ALS IPCA estimator expects data in the format `[rets, Z]` where:
- `rets`: (T, N) array of returns
- `Z`: (T, N, L) array of characteristics

In [ ]:
# Prepare data: transpose X from (N, T) to (T, N) and characteristics from (N, T, L) to (T, N, L)
rets = sim_data['X'].T  # (T, N)
Z = np.transpose(sim_data['characteristics'], (1, 0, 2))  # (T, N, L)

data = [rets, Z]

print(f"Returns shape: {rets.shape}")
print(f"Characteristics shape: {Z.shape}")

---
## 4. Fit ALS IPCA Model

In [ ]:
model = ALSIPCA(num_assets=N, num_fact=K, num_charact=L, win_len=T)

Gamma_hat, objective_history = model.fit(
    data,
    max_iter=500,
    tol=1e-8,
    verbose=True,
    seed=SEED
)

In [ ]:
results = model.get_results()

print(f"\nConverged in {results['n_iterations']} iterations")
print(f"Initial objective: {objective_history[0]:.6f}")
print(f"Final objective: {objective_history[-1]:.6f}")
print(f"Reduction: {(1 - objective_history[-1]/objective_history[0])*100:.2f}%")

---
## 5. Convergence Analysis

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Objective function
axes[0].plot(objective_history, 'b-', lw=1.5)
axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('Objective Function')
axes[0].set_title('ALS IPCA Convergence')
axes[0].axhline(objective_history[-1], color='r', ls='--', alpha=0.5, label=f'Final: {objective_history[-1]:.4f}')
axes[0].legend()

# Log scale
axes[1].semilogy(objective_history, 'b-', lw=1.5)
axes[1].set_xlabel('Iteration')
axes[1].set_ylabel('Objective Function (log scale)')
axes[1].set_title('Convergence (Log Scale)')

plt.tight_layout()
plt.show()

---
## 6. Compare Estimated vs True Gamma

IPCA identifies $\Gamma$ only up to a rotation matrix $H$, so we use Procrustes alignment to find the optimal rotation before comparing.

In [ ]:
error_info = compute_gamma_error(Gamma_true, Gamma_hat)

print("Estimation Error (after Procrustes alignment):")
print(f"  Frobenius norm: {error_info['frobenius_error']:.4f}")
print(f"  RMSE: {error_info['rmse']:.4f}")

In [ ]:
Gamma_hat_aligned = error_info['Gamma_hat_rotated']

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

vmin = min(Gamma_true.min(), Gamma_hat_aligned.min())
vmax = max(Gamma_true.max(), Gamma_hat_aligned.max())

# True Gamma
im1 = axes[0].imshow(Gamma_true, cmap='RdBu_r', aspect='auto', vmin=vmin, vmax=vmax)
axes[0].set_title('True $\\Gamma$')
axes[0].set_xlabel('Factor')
axes[0].set_ylabel('Characteristic')
plt.colorbar(im1, ax=axes[0])

# Estimated Gamma
im2 = axes[1].imshow(Gamma_hat_aligned, cmap='RdBu_r', aspect='auto', vmin=vmin, vmax=vmax)
axes[1].set_title('Estimated $\\hat{\\Gamma}$ (aligned)')
axes[1].set_xlabel('Factor')
axes[1].set_ylabel('Characteristic')
plt.colorbar(im2, ax=axes[1])

# Difference
diff = Gamma_true - Gamma_hat_aligned
vabs = max(abs(diff.min()), abs(diff.max()))
im3 = axes[2].imshow(diff, cmap='RdBu_r', aspect='auto', vmin=-vabs, vmax=vabs)
axes[2].set_title('Difference')
axes[2].set_xlabel('Factor')
axes[2].set_ylabel('Characteristic')
plt.colorbar(im3, ax=axes[2])

plt.tight_layout()
plt.show()

In [ ]:
# Scatter plot: True vs Estimated
fig, ax = plt.subplots(figsize=(7, 7))

gamma_true_flat = Gamma_true.flatten()
gamma_hat_flat = Gamma_hat_aligned.flatten()

ax.scatter(gamma_true_flat, gamma_hat_flat, s=80, alpha=0.7, edgecolors='black')

lims = [min(gamma_true_flat.min(), gamma_hat_flat.min()) - 0.1,
        max(gamma_true_flat.max(), gamma_hat_flat.max()) + 0.1]
ax.plot(lims, lims, 'r--', lw=2, label='45° line')

ax.set_xlabel('True $\\Gamma$ elements')
ax.set_ylabel('Estimated $\\hat{\\Gamma}$ elements')
ax.set_title('True vs Estimated Gamma (Procrustes Aligned)')
ax.legend()
ax.set_xlim(lims)
ax.set_ylim(lims)
ax.set_aspect('equal')

corr = np.corrcoef(gamma_true_flat, gamma_hat_flat)[0, 1]
ax.text(0.05, 0.95, f'Correlation: {corr:.4f}', transform=ax.transAxes,
        fontsize=12, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat'))

plt.tight_layout()
plt.show()

---
## 7. Compare Estimated vs True Factors

In [ ]:
F_true = sim_data['factors']  # (T, K)
F_hat = results['factors'].values.T  # (T, K)

# Align factors using the rotation matrix from Gamma alignment
H = error_info['H']
F_hat_aligned = F_hat @ np.linalg.inv(H).T

fig, axes = plt.subplots(K, 1, figsize=(14, 3*K), sharex=True)
if K == 1:
    axes = [axes]

for k in range(K):
    ax = axes[k]
    ax.plot(F_true[:, k], 'b-', lw=1.5, label='True', alpha=0.8)
    ax.plot(F_hat_aligned[:, k], 'r--', lw=1.5, label='Estimated', alpha=0.8)
    ax.set_ylabel(f'Factor {k+1}')
    ax.legend(loc='upper right')
    
    corr = np.corrcoef(F_true[:, k], F_hat_aligned[:, k])[0, 1]
    ax.text(0.02, 0.95, f'Corr: {corr:.4f}', transform=ax.transAxes,
            fontsize=10, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

axes[-1].set_xlabel('Time')
fig.suptitle('True vs Estimated Factors', y=1.01)
plt.tight_layout()
plt.show()

---
## 8. Summary

In [ ]:
print("=" * 60)
print("SIMULATION STUDY SUMMARY")
print("=" * 60)

print(f"\nSimulation Parameters:")
print(f"  N = {N} entities")
print(f"  T = {T} time periods")
print(f"  K = {K} factors")
print(f"  L = {L} characteristics")
print(f"  Target R² = {TARGET_R2:.0%}")

print(f"\nConvergence:")
print(f"  Iterations: {results['n_iterations']}")
print(f"  Initial objective: {objective_history[0]:.4f}")
print(f"  Final objective: {objective_history[-1]:.4f}")

print(f"\nEstimation Accuracy:")
print(f"  Frobenius error: {error_info['frobenius_error']:.4f}")
print(f"  RMSE: {error_info['rmse']:.4f}")
print(f"  Gamma correlation: {corr:.4f}")

print(f"\nFactor Recovery:")
for k in range(K):
    corr_k = np.corrcoef(F_true[:, k], F_hat_aligned[:, k])[0, 1]
    print(f"  Factor {k+1} correlation: {corr_k:.4f}")

print("=" * 60)